# Práctica guiada - Join con Pandas

# PARTE I

* Algunas de las operaciones más interesantes con los datos provienen de la combinación de diferentes fuentes de datos. Estas operaciones pueden ser

    1. simples concatenaciones de datos de datasets diferentes
    2. operaciones más similares a un join o merge en una base de datos

* Tanto `Series` como `DataFrames` fueron construidos teniendo estas operaciones en mente e incluyen funciones y métodos para realizarlas de forma rápida y simple.

* Veremos dos operaciones: `pd.append` y `pd.concat`

In [ ]:
import pandas as pd
import numpy as np

Vamos a crear una función que crea un `DataFrame` para simplificar algunos pasos:

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

También vamos a crear una clase para poder imprimir "side by side" diferentes `DataFrame`. La idea es poder visualizar algunas de las operaciones que vamos a realizar.

In [ ]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

## Concatenación simple con ``pd.concat``

* ``pd.concat()``, permite hacer concatenaciones simples de diferentes `Series`.

* Tiene una sintaxis similar a su análoga en Numpy ``np.concatenate``. Pero contiene algunas opciones más:

```python
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

In [ ]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

También permite concater objetos de mayor dimensionalidad como `DataFrame`:

In [ ]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')

* Por defecto, la concatenación se hace en el sentido de las filas del ``DataFrame`` (i.e., ``axis=0``).
* Puede especificarse el eje sobre cual realizar la concatenación:

In [ ]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='columns')")

### Índice duplicados

Recordemos que numpy también presenta un método de concatenación.

Una diferencia importante entre  ``np.concatenate`` y ``pd.concat`` es que la concatenación de Pandas preserva los índices, incluso si el resultado implica índices duplicados:

In [ ]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # índices duplicados!
display('x','y')

In [ ]:
display('x', 'y', 'pd.concat([x, y])')

Si bien `DataFrame` permite índices duplicados es preferible eviatarlos.

#### Verificando la existencia índices duplicados

* Podemos verificar si existen índices solapados en el resultado de ``pd.concat()`` usando una ``verify_integrity`` flag.

* Al setear esto en True, la concatenación arrojará una excepción si existe algún índice duplicado:

In [ ]:
# El parámetro verify_integrity evita que concatenemos dataframes con índices iguales.
pd.concat([x, y], verify_integrity=True)

In [ ]:
# En general, para no recibir este comportamiento frente a un error, podemos generar una excepción para mostrarselo
# al usuario de forma más prolija
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

#### Ignorando el índice

* En algunos casos el índice no importa u podemos ignorarlo: simplemente usamos nuevamente ``ignore_index``. Al setearlo en True, se genera un nuevo índice.

In [ ]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

### El método ``append()``

* Dado que la concatenación de arrays es bastante común, ``Series`` y ``DataFrame``tienen un método ``append`` 

* Por ejemplo, en lugar de llamar ``pd.concat([df1, df2])``, es posible llamar al más sencillo ``df1.append(df2)``:

In [ ]:
display('df1', 'df2', 'df1.append(df2)')

In [ ]:
df1.append(df2)

In [ ]:
df1

* **IMPORTANTE:** tener en cuenta que, a diferencia del método `append` de listas, el `append()` de Pandas no modifica el objeto original. Genera un nuevo objeto con los datos combinados.
* Dado que esto implica crear un índice nuevo y un nuevo dataset `append` puede no ser el mejor método si se planea concatenar muchos datasets seguidos.
* En estos casos es mejor usar la función `pd.concat()`.

# PARTE II 

## Tipos de relaciones

Una de las características más valiosas de la librería Pandas es su funcionalidad para realizar joins en memoria y de manera eficiente.

El método merge() permite trabajar con objetos que presentan distintos tipos de relaciones:
 1. Uno a uno
 2. Muchos a uno
 3. Muchos a muchos
 

### 1. Join uno a uno

In [ ]:
import pandas as pd
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1)
print(df2)

In [ ]:
# Vemos que cada empleado tiene un sólo grupo y una sola fecha de contratación.
# Combinamos los dataframes usando pd.merge()
# Notar que la función merge encontró la única columna en común entre los dos dataframe ("employee"). 
# La función requiere que la columna tenga el mismo nombre en los dos df.
df3 = pd.merge(df1, df2)
df3

### 2. Join uno a muchos

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
print(df4)

In [ ]:
# Notar que cada supervisor pertenece a UN grupo que puede tener MUCHOS empleados.
# En el join entre empleados y supervisores, los empleados aparecerán una sola vez pero los supervisores
# pueden repetirse.
print (pd.merge(df3, df4))

### 3. Join muchos a muchos

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
print(df5)
print(df1)

In [ ]:
# Notar que cada grupo tiene MUCHOS skills asociados y a su vez pueden pertenecer a él MUCHOS empleados
# Por lo tanto el join entre la tabla de skills y la de empleados es de MUCHOS a MUCHOS.
# Vamos a ver en el resultado que tanto los skills como los empleados pueden repetirse.
df6 = pd.merge(df1, df5)
print(df6)

## Joins por diferentes columnas

Puede suceder que en nuestros dataframes no haya una única columna con el mismo nombre en ambas tablas para poder hacer el join. 
Para resolver este problema, Pandas implementa los parámetros "on", "right_on" y "left_on" donde podemos especificar por cuáles columnas vamos a unir los datos. 

### 1. Join con "on"

In [ ]:
print(df1)
print(df2)

In [ ]:
pd.merge(df1, df2, on='employee')

### 2. Join con "left_on" y "right_on"

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df1)
print(df3)

In [ ]:
pd.merge(df1, df3, left_on="employee", right_on="name")

### 3. Join por más de una columna

In [ ]:
df7 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization'],
                    'tools':  ['calculator','desktop computer','laptop computer','server','desktop computer','board']
                    })
print(df6)
print(df7)

In [ ]:
# Ahora podemos ver las herramientas por empleado.

pd.merge(df6,df7,left_on=['group','skills'], right_on = ['group','skills'])

In [ ]:
# Como los nombres de las columnas son iguales, utilizar sólo "on" es equivalente.
pd.merge(df6,df7,on=['group','skills'])

## Tipos de joins

### 1. Left joins

In [ ]:
import pandas as pd
from IPython.display import display
from IPython.display import Image
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a

In [ ]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

In [ ]:
# Utilizando el valor left en la forma del join produce una lista completa de las filas de df_a, 
# con las filas que matchean de df_b. Si no hay matcheo, las columnas que vienen de df_b serán nulas:
pd.merge(df_a, df_b, on='subject_id', how='left')

### Check: Cual sería el resultado de cambiar left por right?

In [ ]:
pd.merge(df_a, df_b, on='subject_id', how='right')

### 2. Inner y outer join

In [ ]:
# Como mencionamos antes utilizar la forma outer (OUTER JOIN) 
# produce un conjunto de todas las filas en df_a o df_b. 
# Todas las columnas tendran valores si la fila de un lado tiene su correspondiente en el otro.
# Si no hya matcheo las columnas del que no había valor se completan con null.
pd.merge(df_a, df_b, on='subject_id', how='outer')

### Check: Qué pasaría si usamos un inner join?

In [ ]:
pd.merge(df_a, df_b, on='subject_id', how='inner')

### Encontrando los casos que se aparecen en un dataframe pero no en otro con left join.

Un problema común que podemos querer resolver es encontrar los casos que se encuentran en una tabla pero no en otra. Esto se puede hacer fácilmente con un left join por el o los campos que conforman la clave. 


In [ ]:
df1 = pd.DataFrame(data = {'col1' : [1, 2, 3, 4, 5], 'col2' : [10, 11, 12, 13, 14]}) 
df2 = pd.DataFrame(data = {'col1' : [1, 2, 3], 'col2' : [10, 11, 12]})
print(df1)
print(df2)

In [ ]:
# Agrego en ambos dataframes unas claves que toman siempre el mismo valor para después poder compararlas.
df1['key1'] = 1
df2['key2'] = 1
print(df1)
print(df2)

In [ ]:
# Cuando hacemos el left join, los valores de key2 se llenan con null para los valores de df2 que no existen en df1
df1 = pd.merge(df1, df2, on=['col1', 'col2'], how = 'left')
df1

In [ ]:
# Subseteo el resultado del merge para quedarme con los que aparecen en df1 pero no en df2.
df3 = df1[~(df1.key2 == df1.key1)]
df3 = df3.drop(['key1','key2'], axis=1)
df3

### Revisión de buenas prácticas de performance

El método join, cuenta con la misma sintaxis y posibilidades que el método merge pero con la diferencia de que siempre hace la relación por el index. 

In [ ]:
# Creamos dos Dataframes de tamaño 1000000
df1 = pd.DataFrame(np.arange(1000000), columns=['A'])
df1['B'] = np.random.randint(0,1000,(1000000))
df1.head()

In [ ]:
df2 = pd.DataFrame(np.arange(1000000), columns=['A2'])
df2['B2'] = np.random.randint(0,1000,(1000000))
df2.head()

In [ ]:
# Medimos el tiempo de ejecución del merge
def a():
    x = df1.merge(df2, how='left', left_on='A', right_on='A2')

In [ ]:
%timeit a()

In [ ]:
# Medimos el tiempo de ejecución del join
def b():
    x = df1.set_index('A').join(df2.set_index('A2'), how='left')

In [ ]:
%timeit b()

In [ ]:
# Creamos dos Dataframes de tamaño 1000000
df1 = pd.DataFrame(np.arange(1000000), columns=['A'])
df1['B'] = np.random.randint(0,1000,(1000000))
df2 = pd.DataFrame(np.arange(1000000), columns=['A2'])
df2['B2'] = np.random.randint(0,1000,(1000000))

In [ ]:
def set_indA(df):
     df.set_index('A')

In [ ]:
def set_indA2(df):
    df.set_index('A2')

In [ ]:
def c():
    df1.join(df2)

In [ ]:
%timeit set_indA(df1)

In [ ]:
%timeit set_indA2(df2)

In [ ]:
%timeit c()

 ##  Trabajo con series de tiempo: calcular variaciones con shift()

Cuando se trabaja con series de tiempo, es muy habitual trabajar agregar al dataframe una variable que indique cuánto se modificó determinado valor desde la última medición.

In [ ]:
#Creamos el dataframe
df= pd.DataFrame()
# Agregamos las series de datos
df['hora']=['10:30','10:31','10:38','10:40','10:41']
df['cantidad']=[20,20,9,12,12]
df

In [ ]:
df['cantidad_anterior'] = df['cantidad'].shift(1)
df

In [ ]:
df['variacion'] = df['cantidad'] - df['cantidad_anterior']


In [ ]:
df

### Variaciones con agrupamiento

Es muy habitual tener que calcular esto mismo pero por cada individuo o categoría que interesa estudiar 

In [ ]:
df=pd.DataFrame()
df['operador']=['Q8','Q8','Q8','Q7','Q9','Q9']
df['hora']=['10:30','10:31','10:32','10:38','10:40','10:41']
df['cantidad']=[15,20,10,9,12,12]
df


In [ ]:
df['cantidad_ant_operador'] = df.groupby(['operador'])['cantidad'].transform(lambda x:x.shift())

In [ ]:
df

In [ ]:
df['variacion_operador'] = df['cantidad'] - df['cantidad_ant_operador']
df